In [ ]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import last,max

In [ ]:
spark = SparkSession.builder.appName("dieseaseStream").getOrCreate()

In [ ]:
#line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.142").option("port",9000).load()

In [ ]:
#point = line.select(explode(split(line.value," ")).alias("lon"))
#point = point.withColumn('lat', split(point["lon"], '\,')[1])
#point = point.withColumn('lon', split(point["lon"], '\,')[0])
#pointCount = point.groupBy('lat','lon').count()

In [ ]:
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('lat', split(ppoint["id"], '\,')[3].cast(types.DoubleType()))
ppoint = ppoint.withColumn('lon', split(ppoint["id"], '\,')[2].cast(types.DoubleType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
latest = ppoint.groupby('id').agg(max('timestamp'), last('lat'), last('lon')).toDF('id','time','lat','lon').drop('id','time')

In [ ]:
query = latest.writeStream.outputMode('complete').format("console").start()
query.awaitTermination()